In [1]:
import pandas as pd
import numpy as np
import collections
import copy
import random
import re
import matplotlib.pyplot as plt
from nlp.utils import (
    plot_histogram,
    get_completetext,
    plot_wordcloud,
    print_statistics,
    groups_frequency_sort)
from nlp.text_statistics import (
    count_tokens,
    unique_tokens
)
from utils.read_files import (
    get_items)
from item.item_list import (
    ItemList,
    Item
)
from item.utils import get_tokens_set
from item.clustering.utils import *

# Load dataset

In [2]:
# It gets the descriptions processed:
itemlist = ItemList()
itemlist.load_items_from_file('../data/items_preprocessed_notafiscal.csv.zip')
# itemlist.load_items_from_file('../data/items_preprocessed_complete_druid_v2.csv.zip')

In [3]:
itemlist.items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3679117 entries, 0 to 3679116
Data columns (total 15 columns):
 #   Column              Dtype  
---  ------              -----  
 0   palavras            object 
 1   unidades_medida     object 
 2   numeros             object 
 3   cores               object 
 4   materiais           object 
 5   tamanho             object 
 6   quantidade          object 
 7   preco               float64
 8   dsc_unidade_medida  object 
 9   original            object 
 10  original_prep       object 
 11  ano                 int64  
 12  licitacao           int64  
 13  licitacao_item      int64  
 14  item_id             int64  
dtypes: float64(1), int64(4), object(10)
memory usage: 421.0+ MB


In [4]:
itemlist.items_df

,palavras,unidades_medida,numeros,cores,materiais,tamanho,quantidade,preco,dsc_unidade_medida,original,original_prep,ano,licitacao,licitacao_item,item_id
0,"['prestacao', 'servico', 'eletrico', 'veiculo'...",[],['2.41e+03'],[],[],[],[],63.00,hora,PRESTACAO DE SERVICOS ELETRICOS EM VEICULOS LE...,"['prestacao', 'servico', 'eletrico', 'veiculo'...",2014,91611180,38706974,0
1,"['oleo', 'diesel', 'combustivel']",[],"['1.00e+01', '2.57e+03']",[],[],[],[],2.71,litro,OLEO DIESEL COMBUSTIVEL S10 2567,"['oleo', 'diesel', 'combustivel', '10', '2567']",2014,91611231,38706975,1
2,"['graxo', 'brif', 'ngl2']",[],"['9.00e+00', '2.80e+03']",[],[],[],[],30.00,unidade,G 052737Q9 GRAXA LUBRIF NGL2 2797,"['g', 'q', '9', 'graxo', 'brif', 'ngl2', '2797']",2014,91638555,38706976,2
3,"['oleo', 'dif']",[],"['9.00e+00', '8.50e+01', '1.40e+02', '2.80e+03']",[],[],[],[],25.00,unidade,G 052815Q9 OLEO DIF 85W140 2798,"['g', 'q', '9', 'oleo', 'dif', '85', 'w', '140...",2014,91638555,38706977,3
4,['pneu'],[],"['1.30e+01', '4.72e+02']",[],[],[],[],180.00,unidade,PNEU 16570R13 472,"['pneu', 'r', '13', '472']",2014,91611257,38706978,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3679112,"['servico', 'mecanico', 'motor', 'freio']",[],['7.00e+03'],[],[],[],[],75.00,hora,SERVICOS MECANICOS DE MOTOR E FREIOS 7001,"['servico', 'mecanico', 'motor', 'freio', '7001']",2020,121762479,44934348,3679112
3679113,"['contratacao', 'software', 'adiministracao', ...",[],['2.45e+03'],[],[],[],[],6300.00,mes,CONTRATACAO DE SOFTWARE DE ADIMINISTRACAO PUBL...,"['contratacao', 'software', 'adiministracao', ...",2020,121762480,44934349,3679113
3679114,"['escada', 'extensivo', 'degrau', 'rebitar']",[],"['3.60e+02', '6.00e+02', '8.74e+03']",[],[],[],[],588.00,unidade,ESCADA EXTENSIVA 360 X 600 DEGRAU REBITADO 8740,"['escada', 'extensivo', '360', 'x', '600', 'de...",2020,121762481,44934350,3679114
3679115,"['oleo', 'diesel', 'combustivel', 'comum']",[],['4.92e+02'],[],[],[],[],3.75,litro,OLEO DIESEL COMBUSTIVEL COMUM 492,"['oleo', 'diesel', 'combustivel', 'comum', '492']",2020,121762482,44934351,3679115


# Load items

In [5]:
combustiveis = ['gasolina', 'alcool', 'querosene', 'oleo', 'etanol', 'diesel']
hosp_medicamentos = ['vitamina', 'acido', 'lorazepam', 'luva', 'mascara', 'algodao', 'dipirona', 'sabonete', 'sabao', \
                     'pen', 'atenolol', 'alopurinol', 'dea', 'estetoscopio', 'maca', 'reanimador', 'esfigmomanometro', \
                     'oximetro', 'camisola', 'respirador']
pecas = ['bateria', 'filtro', 'componentes', 'capota', 'macaco', 'pastilha', 'espelho', 'antena', 'carroceria', 'lampada',
         'pneu', 'freio', 'parabrisa', 'parachoque', 'roda', 'lataria', 'calota', 'ignicao', 'amortecedor', 'coxim',
         'radiador', 'virabrequim', 'correia', 'embreagem', 'locacao']
alimentos = ['feijao', 'arroz', 'leite', 'farinha', 'acucar', 'carne', 'macarrao']

In [6]:
sample = combustiveis + hosp_medicamentos + pecas + alimentos

## Items table

In [4]:
items = pd.read_csv('../data/output/druid_fasttext/items_clusters_train_wo_out.csv.zip', sep=';', low_memory=False)

In [5]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8746361 entries, 0 to 8746360
Data columns (total 12 columns):
 #   Column              Dtype  
---  ------              -----  
 0   grupo               object 
 1   dsc_unidade_medida  object 
 2   grupo_id            int64  
 3   item_id             int64  
 4   grupo_ruido         int64  
 5   item_ruido          int64  
 6   preco               float64
 7   original            object 
 8   licitacao           int64  
 9   ano                 int64  
 10  original_dsc        object 
 11  primeiro_termo      object 
dtypes: float64(1), int64(6), object(5)
memory usage: 800.8+ MB


In [8]:
len(sample)

58

In [9]:
items_sample_df = items.loc[items['primeiro_termo'].isin(sample)]

In [10]:
len(items_sample_df)

860675

In [11]:
items_id = list(items_sample_df['item_id'])

## Groups items

In [7]:
results, outliers = load_clustering_results_pickle('../data/output/nota_fiscal_fasttext/')

In [8]:
len(results)

35829

In [9]:
sample = set(sample)

In [10]:
items_id = []

for group, ids in results.items():
    group = group.split('_')
    first_token = group[0]
    
    if first_token in sample:
        items_id += ids

for group, ids in outliers.items():
    group = group.split('_')
    first_token = group[0]
    
    if first_token in sample:
        items_id += ids

In [11]:
len(items_id)

682493

In [12]:
items_ = itemlist.items_df.loc[items_id]

In [13]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
items_.head(1000)

,palavras,unidades_medida,numeros,cores,materiais,tamanho,quantidade,preco,dsc_unidade_medida,original,original_prep,ano,licitacao,licitacao_item,item_id
766,"['arroz', 'rei', 'panela', 't1']",['kg'],['5.00e+00'],[],[],[],[],10.6900,unidade,ARROZ REI DA PANELA T1 5KG,"['arroz', 'rei', 'panela', 't1', '5', 'kg']",2014,92231427,38462589,766
847,"['arroz', 'rei', 'panela', 't1']",['kg'],['5.00e+00'],[],[],[],[],10.6900,unidade,ARROZ REI DA PANELA T1 5KG,"['arroz', 'rei', 'panela', 't1', '5', 'kg']",2014,92231415,38462752,847
1597,"['arroz', 'rei', 'panela', 't1']",['kg'],['5.00e+00'],[],[],[],[],10.6900,unidade,ARROZ REI DA PANELA T1 5KG,"['arroz', 'rei', 'panela', 't1', '5', 'kg']",2014,92231367,38785226,1597
1627,"['arroz', 'rei', 'panela']",['kg'],"['1.00e+00', '5.00e+00']",[],[],[],[],9.0000,unidade,ARROZ REI DA PANELA T 1 5KG,"['arroz', 'rei', 'panela', 't', '1', '5', 'kg']",2014,92231365,38785259,1627
1819,"['arroz', 'rei', 'panela', 't1']",['kg'],['5.00e+00'],[],[],[],[],10.6900,unidade,ARROZ REI DA PANELA T1 5KG,"['arroz', 'rei', 'panela', 't1', '5', 'kg']",2014,92231405,38738804,1819
2052,"['arroz', 'rei', 'panela', 't1']",['kg'],['5.00e+00'],[],[],[],[],10.6900,unidade,ARROZ REI DA PANELA T1 5KG,"['arroz', 'rei', 'panela', 't1', '5', 'kg']",2014,92231394,38743823,2052
2088,"['arroz', 'rei', 'panela', 't1']",['kg'],['5.00e+00'],[],[],[],[],10.6900,unidade,ARROZ REI DA PANELA T1 5KG,"['arroz', 'rei', 'panela', 't1', '5', 'kg']",2014,92231392,38743859,2088
2430,"['arroz', 'rei', 'panela']",['kg'],"['1.00e+00', '5.00e+00']",[],[],[],[],9.0000,unidade,ARROZ REI DA PANELA T 1 5KG,"['arroz', 'rei', 'panela', 't', '1', '5', 'kg']",2014,92231350,38608492,2430
2536,"['arroz', 'rei', 'panela', 't1']",['kg'],['5.00e+00'],[],[],[],[],10.6900,unidade,ARROZ REI DA PANELA T1 5KG,"['arroz', 'rei', 'panela', 't1', '5', 'kg']",2014,92231348,38608564,2536
2707,"['arroz', 'rei', 'panela', 't1']",['kg'],['5.00e+00'],[],[],[],[],10.6900,unidade,ARROZ REI DA PANELA T1 5KG,"['arroz', 'rei', 'panela', 't1', '5', 'kg']",2014,92231344,38609068,2707


In [14]:
items_['licitacao_item'] = range(len(items_))

In [15]:
items_ = items_[['licitacao', 'licitacao_item', 'original', 'dsc_unidade_medida', 'preco', 'ano']].rename(columns={})

In [16]:
items_ = items_.rename(columns={'licitacao': 'seq_dim_licitacao', 'licitacao_item': 'seq_dim_item', 'original': 'nom_item',
                                'preco': 'vlr_unitario_homologado', 'ano': 'ano_exercicio'})

In [17]:
items_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 682493 entries, 766 to 3334695
Data columns (total 6 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   seq_dim_licitacao        682493 non-null  int64  
 1   seq_dim_item             682493 non-null  int64  
 2   nom_item                 682493 non-null  object 
 3   dsc_unidade_medida       682391 non-null  object 
 4   vlr_unitario_homologado  682493 non-null  float64
 5   ano_exercicio            682493 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 36.4+ MB


In [18]:
items_.head()

,seq_dim_licitacao,seq_dim_item,nom_item,dsc_unidade_medida,vlr_unitario_homologado,ano_exercicio
766,92231427,0,ARROZ REI DA PANELA T1 5KG,unidade,10.69,2014
847,92231415,1,ARROZ REI DA PANELA T1 5KG,unidade,10.69,2014
1597,92231367,2,ARROZ REI DA PANELA T1 5KG,unidade,10.69,2014
1627,92231365,3,ARROZ REI DA PANELA T 1 5KG,unidade,9.00,2014
1819,92231405,4,ARROZ REI DA PANELA T1 5KG,unidade,10.69,2014


In [19]:
items_.to_csv('../data/items_sample.csv', sep=';', index=False)